In [1]:

# Step 1: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from googletrans import Translator

In [20]:
df = pd.read_csv('Beijing.csv', encoding='gbk')
df = df.rename(columns={"日期": "date", "最高温度": "max_temp", "最低温度": "min_temp", "天气": "weather", "风力风向": "wind", "空气质量": "air_quality"})
df.head()

,date,max_temp,min_temp,weather,wind,air_quality
0,2022-01-01 周六,19,10,多云,东北风2级,64 良
1,2022-01-02 周日,23,12,多云,北风2级,62 良
2,2022-01-03 周一,25,12,多云,东风1级,74 良
3,2022-01-04 周二,25,15,晴~阴,东南风2级,88 良
4,2022-01-05 周三,24,13,多云,东北风1级,70 良


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         730 non-null    object
 1   max_temp     730 non-null    int64 
 2   min_temp     730 non-null    int64 
 3   weather      730 non-null    object
 4   wind         730 non-null    object
 5   air_quality  730 non-null    object
dtypes: int64(2), object(4)
memory usage: 34.3+ KB


In [22]:
# convert date to datetime
df['date'] = df['date'].str.strip(' 周六	 周日 周一 周二 周三 周四 周五')
df['date'] = pd.to_datetime(df['date'], format='mixed')


In [23]:
df.drop(['weather', 'wind', 'air_quality'], axis=1, inplace=True)


In [24]:
df.describe()

,date,max_temp,min_temp
count,730,730.000000,730.000000
mean,2022-07-01 23:59:59.999999744,22.567123,13.563014
min,2022-01-01 00:00:00,-5.000000,-12.000000
25%,2022-04-02 00:00:00,16.000000,7.000000
50%,2022-07-02 00:00:00,26.000000,16.000000
75%,2022-10-01 00:00:00,31.000000,23.000000
max,2022-12-31 00:00:00,38.000000,28.000000
std,NaN,10.361981,10.501003


In [25]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pmdarima import auto_arima

In [26]:
train_df = df.copy()

# split the data into train and test
train = train_df[:len(train_df)-50]
test = train_df[len(train_df)-50:]

In [27]:
# fit auto_arima model on train data to find the best parameters
auto_arima_model = auto_arima(train['max_temp'],
                       m=12,
                       d=0,
                       D=0,
                       max_order=None,                       
                       max_p=7,
                       max_q=7,
                       max_d=2,
                       max_P=4,
                       max_Q=4,
                       max_D=2,
                       maxiter = 50,
                       alpha = 0.05,
                       n_jobs = -1,
                       seasonal=True,
                       trace=True,
                       error_action='ignore',  
                       suppress_warnings=True, 
                       stepwise=True
                      )

auto_arima_model.summary()

/home/elijah/Documents/DataProjects/timeseries/env/lib/python3.10/site-packages/pmdarima/arima/_validation.py:76: UserWarning: stepwise model cannot be fit in parallel (n_jobs=1). Falling back to stepwise parameter search.
  warnings.warn('stepwise model cannot be fit in parallel (n_jobs=%i). '


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[12] intercept   : AIC=inf, Time=10.53 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=4986.585, Time=0.07 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=3511.192, Time=3.67 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=4180.019, Time=2.69 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=6344.017, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=3510.395, Time=0.14 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=3511.289, Time=1.78 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=3510.558, Time=9.70 sec
 ARIMA(2,0,0)(0,0,0)[12] intercept   : AIC=3506.381, Time=0.34 sec
 ARIMA(2,0,0)(1,0,0)[12] intercept   : AIC=3507.532, Time=5.97 sec
 ARIMA(2,0,0)(0,0,1)[12] intercept   : AIC=3507.577, Time=2.69 sec
 ARIMA(2,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=8.71 sec
 ARIMA(3,0,0)(0,0,0)[12] intercept   : AIC=3505.014, Time=0.68 sec
 ARIMA(3,0,0)(1,0,0)[12] intercept   : AIC=3506.409, Time=11.93 sec
 ARIMA(3,0,0)(0,0,1)[12] in

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  680
Model:               SARIMAX(2, 0, 1)   Log Likelihood               -1734.775
Date:                Fri, 05 May 2023   AIC                           3479.550
Time:                        16:06:50   BIC                           3502.160
Sample:                             0   HQIC                          3488.302
                                - 680                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0682      0.052      1.315      0.189      -0.033       0.170
ar.L1          1.6672      0.056     29.744      0.000       1.557       1.777
ar.L2         -0.6702      0.055    -12.221      0.000      -0.778      -0.563
ma.L1         -0.8786      0.039    -22.402      0.000      -0.955      -0.802
sigma2         9.5915      0.398     24.092      0.000       8.811      10.372
===================================================================================
Ljung-Box (L1) (Q):                   0.22   Jarque-Bera (JB):               228.94
Prob(Q):                              0.64   Prob(JB):                         0.00
Heteroskedasticity (H):               1.20   Skew:                            -0.79
Prob(H) (two-sided):                  0.17   Kurtosis:                         5.36
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [28]:
arima_model = ARIMA(train['max_temp'], order=(2, 0, 1), seasonal_order=(0, 0, 0, 12))
arima_model = arima_model.fit()

In [29]:
# predict the values on test data
predictions = arima_model.predict(start=len(train), end=len(train)+len(test)-1, typ='levels').rename('ARIMA Predictions')

# print the mean squared error and mean absolute error
print('Mean Squared Error:', mean_squared_error(test['max_temp'], predictions))
print('Mean Absolute Error:', mean_absolute_error(test['max_temp'], predictions))

Mean Squared Error: 163.62911050975464
Mean Absolute Error: 11.277165122321383


In [42]:
# predict max_temp for next 30 days
future_dates = [df['date'].iloc[-1] + DateOffset(days=x) for x in range(0, 32)]

# future_dates_df['max_temp']
preds =  arima_model.predict(start=len(df), end=len(df)+31, typ='levels').rename('ARIMA Predictions')

# convert the predictions to dataframe
future_dates_df = pd.DataFrame(
    {'date': future_dates,
        'max_temp': preds}
)

future_dates_df


,date,max_temp
730,2022-12-31,17.783472
731,2023-01-01,17.826930
732,2023-01-02,17.869979
733,2023-01-03,17.912622
734,2023-01-04,17.954863
735,2023-01-05,17.996706
736,2023-01-06,18.038155
737,2023-01-07,18.079212
738,2023-01-08,18.119883
739,2023-01-09,18.160171
